In [243]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from IPython.display import clear_output

url_list = []
database_url = "https://www.jefit.com/exercises/bodypart.php?id=11&exercises=All&All=0&Bands=0&Bench=0&Dumbbell=0&EZBar=0&Kettlebell=0&MachineStrength=0&MachineCardio=0&Barbell=0&BodyOnly=0&ExerciseBall=0&FoamRoll=0&PullBar=0&WeightPlate=0&Other=0&Strength=0&Stretching=0&Powerlifting=0&OlympicWeightLifting=0&Beginner=0&Intermediate=0&Expert=0&page="

page_prefix_url = 'https://www.jefit.com/exercises/'

## Page 1~130
for i in range(1, 131):
    URL = database_url + str(i)
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, 'html.parser') 
    table = soup.find(id = "hor-minimalist_3").table
    rows = table.find_all('tr')
    for row in rows:
        link = row.find('h4').a.attrs['href']
        url_list.append(page_prefix_url+link)
        
url_list

['https://www.jefit.com/exercises/1053/1-Leg-Pushup',
 'https://www.jefit.com/exercises/465/90-90-Hamstring',
 'https://www.jefit.com/exercises/56/Ab-Crunch-Machine',
 'https://www.jefit.com/exercises/670/Ab-Draw-Leg-Slide',
 'https://www.jefit.com/exercises/967/Abdominal-Pendulum',
 'https://www.jefit.com/exercises/308/Aerobics',
 'https://www.jefit.com/exercises/57/Air-Bike',
 'https://www.jefit.com/exercises/464/All-Fours-Squad-Stretch',
 'https://www.jefit.com/exercises/58/Alternate-Heel-Touchers',
 'https://www.jefit.com/exercises/610/Alternate-Leg-Bridge',
 'https://www.jefit.com/exercises/892/Alternate-Leg-Diagonal-Bound',
 'https://www.jefit.com/exercises/734/Alternate-Leg-Reverse-Hyper-on-Flat-Bench',
 'https://www.jefit.com/exercises/968/Alternate-Reach-and-Catch',
 'https://www.jefit.com/exercises/626/Alternating-Arm-Cobra',
 'https://www.jefit.com/exercises/289/Ankle-Circles',
 'https://www.jefit.com/exercises/384/Ankle-On-The-Knee',
 'https://www.jefit.com/exercises/867/Ar

In [244]:
col = ['Name', 'Main Muscle Group', 'Detailed Muscle Group', 'Type', 'Mechanics', 'Equipment', 'Difficulty', 'Guide']
df = pd.DataFrame(columns = col)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
Name                     0 non-null object
Main Muscle Group        0 non-null object
Detailed Muscle Group    0 non-null object
Type                     0 non-null object
Mechanics                0 non-null object
Equipment                0 non-null object
Difficulty               0 non-null object
Guide                    0 non-null object
dtypes: object(8)
memory usage: 0.0+ bytes


In [245]:
count = 0
total = len(url_list)
for page_url in url_list:
    final_body_list = []
    rq = requests.get(page_url) 
    soup = BeautifulSoup(rq.content, 'html.parser')
    content = soup.find("div", {"class": "col-sm-9 order-sm-2"})
    # Get the title
    title = content.find("div", {"class": "mt-2 border-bottom"}).get_text()[1: -8]
    final_body_list.append(title)

    # Get the exercise details
    bodys = content.find("div", {"class": "mt-2 p-2"})
    body_list = []
    for string in bodys.stripped_strings:
        body_list.append(repr(string)[1:-1])
    order = [1,3,5,7,9,11]

    for i in order:
        final_body_list.append(body_list[i])
    # Get the exercise guide
    stuff = content.find_all("div", {"class": "col-sm-6 mt-2"})
    
    if (len(stuff) == 3):
        guide = stuff[2].p
    else:
        guide = content.find("div", {"class": "col-sm-12 mt-2"}).p 
    
    #guide = content.find(lambda tag: tag.name == 'div' and tag.get('class') == ['p-2'])
    # print(title)
    final_body_list.append(guide.get_text()[1:])
    
    #final_body_list
    #print(final_body_list)
    
    df = df.append(pd.Series(final_body_list, index=col), ignore_index=True)
    
    count = count + 1
    if count % 10 ==0:
        clear_output()
        print(str(count/total*100)+'%')

99.30715935334872%


In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 8 columns):
Name                     1299 non-null object
Main Muscle Group        1299 non-null object
Detailed Muscle Group    1299 non-null object
Type                     1299 non-null object
Mechanics                1299 non-null object
Equipment                1299 non-null object
Difficulty               1299 non-null object
Guide                    1299 non-null object
dtypes: object(8)
memory usage: 81.3+ KB


In [247]:
df

,Name,Main Muscle Group,Detailed Muscle Group,Type,Mechanics,Equipment,Difficulty,Guide
0,1 Leg Pushup,Chest,"Shoulders , Triceps",Strength,Compound,Body Only,Beginner,Steps : \n\r\n1.) Start off by getting yoursel...
1,90 90 Hamstring,Upper Legs,Hamstrings,Stretching,N/A,Body Only,Beginner,Steps : \n\r\n1.) To begin this exercise; star...
2,Ab Crunch Machine,Abs,Full,Strength,Isolation,Machine - Strength,Beginner,Steps : \n\r\n1.) Sit down in the machine and ...
3,Ab Draw Leg Slide,Abs,Strength,Isolation,Body Only,Intermediate,+ Add To Routine,Steps :\n\r\n1.) Start off laying on your back...
4,Abdominal Pendulum,Abs,Strength,Isolation,Body Only,Beginner,+ Add To Routine,Steps :\n\r\n1.) Start off by laying with your...
5,Aerobics,Cardio,Cardio,Isolation,Others,Beginner,+ Add To Routine,Aerobic Exercise focuses upon cardiovascular f...
6,Air Bike,Abs,Obliques,Strength,Compound,Body Only,Beginner,Steps : \n\r\n1.) Start off lying flat as if y...
7,All Fours Squad Stretch,Upper Legs,Hamstrings,Stretching,N/A,Body Only,Intermediate,Steps : \n\r\n1.) To begin this exercise; star...
8,Alternate Heel Touchers,Abs,Obliques,Strength,Isolation,Body Only,Beginner,Steps : \n\r\n1.) Start off lying on the floor...
9,Alternate Leg Bridge,Abs,"Glutes , Upper Legs",Strength,Compound,Body Only,Intermediate,Steps : \n\r\n1.) Start by laying on your back...


In [248]:
# Write data into an Excel file
df.to_excel("output.xlsx")

In [250]:
df2 = df
df2.replace('+ Add To Routine', None, inplace=True)
df2.to_excel("exercise_database.xlsx")